In [3]:
import torch
import trimesh
from nerf.provider import NeRFDataset
from nerf.provider_frames import NeRFDatasetFrames
from nerf.utils import *
from nerf.network_time_deform import NeRFNetwork
import argparse

def load_checkpoint(model, checkpoint=None):
        checkpoint_dict = torch.load(checkpoint)
        if 'model' not in checkpoint_dict:
            model.load_state_dict(checkpoint_dict,1)
            log("[INFO] loaded model.")
            return
        missing_keys, unexpected_keys = model.load_state_dict(checkpoint_dict['model'], strict=False)
        print("[INFO] loaded model.")
        return model
    
def load_vertices(coord_file, cuda_id = 0):
    mesh = trimesh.load(coord_file)
    coords =  np.asarray(mesh.vertices)
    coords = torch.from_numpy(coords).cuda(cuda_id)
    coords = coords.to(torch.float32)
    return coords,mesh

In [11]:
num_frames = 2
model_path = '/new_disk/jyh/animationMesh/torch-ngp/results/spider_test/checkpoints/ngp_ep0506.pth.tar'
cuda_id = 1
model = NeRFNetwork(
    encoding="hashgrid", encoding_dir="sphere_harmonics",
    num_layers=2,  geo_feat_dim=15, num_layers_color=3, hidden_dim_color=64,
    num_frames = num_frames,
)
model = load_checkpoint(model, model_path)
model.eval()
model = model.to(cuda_id)
print(next(model.parameters()).device)


[INFO] loaded model.
cuda:1


In [13]:
coord_file = '/new_disk/jyh/datasets/denseView/spider2/dealed_model/%03d.obj' % 50
print(coord_file)
vertices,mesh = load_vertices(coord_file,cuda_id)
vertices = vertices / 3
vertices[:,[2,0,1]] = vertices[:,[0,1,2]]
np.savetxt('hh.txt',vertices.detach().cpu().numpy())
print(vertices)
warp_vertices = model.deform(vertices, 1)
warp_vertices[:,[0,1,2]] = warp_vertices[:,[2,0,1]]
warp_vertices = warp_vertices * 3
print(warp_vertices)
warp_vertices = warp_vertices.detach().cpu().numpy()

_ = mesh.export('before.obj')
mesh.vertices = warp_vertices.tolist()
_ = mesh.export('warp.obj')


/new_disk/jyh/datasets/denseView/spider2/dealed_model/050.obj
tensor([[ 3.4177e-03, -1.2154e-01, -4.4783e-02],
        [-1.1985e-01,  2.3555e-01, -2.4904e-02],
        [-2.3230e-02,  5.7250e-02,  2.4686e-02],
        ...,
        [-1.5323e-01, -2.5637e-01, -9.8321e-03],
        [ 1.4978e-02,  8.1799e-02,  2.1844e-02],
        [-1.5150e-01, -2.2492e-01, -8.5683e-07]], device='cuda:1')
tensor([[-0.1498,  0.0068, -0.3719],
        [-0.0902, -0.3630,  0.6994],
        [ 0.0586, -0.0732,  0.1645],
        ...,
        [-0.0450, -0.4632, -0.7764],
        [ 0.0501,  0.0415,  0.2381],
        [-0.0155, -0.4580, -0.6820]], device='cuda:1', grad_fn=<MulBackward0>)


In [ ]:
tensor([[ 0.1274, -0.3516, -0.0315],
        [-0.1709,  0.1021, -0.0205],
        [-0.1849,  0.1022, -0.0485],
        ...,
        [-0.1469, -0.1623, -0.0382],
        [-0.0052,  0.0927,  0.0194],
        [-0.1565, -0.0637, -0.0284]], device='cuda:1')
tensor([[-0.3102,  0.3950, -0.9807],
        [-0.3035, -0.5141,  0.3920],
        [-0.2025, -0.5316,  0.3875],
        ...,
        [-0.2123, -0.3729, -0.4122],
        [-0.1616,  0.1053,  0.3552],
        [-0.3562, -0.4488, -0.1125]], device='cuda:1', grad_fn=<MulBackward0>)